https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


# *CoastSat*: testing


This example shows users how to extract time-series of shoreline change over the last 30+ years at their site of interest.
There are five main steps:
1. Retrieval of the satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution
3. Intersection of the shorelines with cross-shore transects
4. Tidal correction 
5. Time-series post-processing
6. Validation against in situ surveys

This software is described in details in the following publications: 
- Shoreline detection:                      https://doi.org/10.1016/j.envsoft.2019.104528
- Accuracy assessment and applications:     https://doi.org/10.1016/j.coastaleng.2019.04.004
- Beach slope estimation:                   https://doi.org/10.1029/2020GL088365

## Initial settings

Refer to the **Installation** section of the README for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

## 1. Retrieval of the images from GEE

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided in Tier 1 and Tier 2, only Tier 1 images can be used for time-series analysis.

For Landsat, users can also choose between Collection 1 and Collection 2 with the `collection` variable. Note that from 1st Jan 2022 newly acquired Landsat images are only available in Collection 2, with Landsat 9 only available in Collection 2, so it's preferred that you use Collection 2.

In [2]:
#read a csv file with the coordinates of the sites
sitetable = pd.read_csv('G:/My Drive/MIT/satellite_AI/sat_data/areas_of_interest.csv')
sitetable

,ID,Atoll,Island Name,pt1lat,pt1long,pt2lat,pt2long,pt3lat,pt3long,pt4lat,pt4long,Critical date,Type,Unnamed: 13
0,0,Haa Alifu,Dhihdhoo,6.886873,73.102336,6.895814,73.101447,6.896274,73.114052,6.886438,73.112871,11/1/10,Extension,NaN
1,1,Haa Dhaalu,Kulhudhuffushi,6.614142,73.061912,6.635586,73.058699,6.634345,73.075197,6.617827,73.072117,12/1/10,Extension,NaN
2,2,Shaviyani,Funadhoo,6.154265,73.284247,6.170579,73.285049,6.170008,73.292498,6.153860,73.291068,12/1/18,Extension,NaN
3,3,Shaviyani,Komandoo,6.051581,73.050525,6.056823,73.050907,6.056441,73.057656,6.051108,73.057391,6/1/14,Extension,NaN
4,5,Lhaviyani,Hinnavaru,5.486331,73.407030,5.497300,73.405889,5.497551,73.415926,5.486636,73.416095,6/1/14,Extension,NaN
5,6,Lhaviyani,Naifaru,5.439174,73.361303,5.450183,73.361250,5.450392,73.368619,5.439830,73.369442,6/1/05,Extension,NaN
6,7,Baa,Eydhafushi,5.098853,73.063853,5.107673,73.063995,5.106891,73.077669,5.097289,73.077732,9/1/14,Extension,NaN
7,10,Baa,Thulhaadhoo,5.016289,72.835511,5.027502,72.834448,5.027479,72.845912,5.016243,72.847669,11/1/13,Extension,NaN
8,11,Kaafu,Thulusdhoo,4.366670,73.639003,4.379533,73.639461,4.379576,73.657050,4.367890,73.657758,5/1/16,Extension,NaN
9,13,Kaafu,Huraa,4.327775,73.594801,4.339679,73.595040,4.337569,73.607695,4.326716,73.606778,1/1/19,Extension,NaN


In [3]:
#functions to get the coordinates of a site from the csv file
def get_site_coordinates(sites, site_name):
    site = sites[sites['Island Name'] == site_name]
    polygon =  np.expand_dims(site[['pt1long','pt1lat','pt2long','pt2lat','pt3long','pt3lat','pt4long','pt4lat']].values[0].reshape(4,2), axis=0)
    date = site['Critical date'].values[0]
    return polygon,date

from datetime import timedelta   
def set_date_range(date_string, days_before, days_after):
    date = datetime.strptime(date_string, '%m/%d/%y')
    date_before = date - timedelta(days=days_before)
    date_after = date + timedelta(days=days_after)
    date_before = datetime.strftime(date_before, '%Y-%m-%d')
    date_after = datetime.strftime(date_after, '%Y-%m-%d')
    return [date_before, date_after]

In [17]:
#functions to check or download the images
def batch_download(sitetable, site_list, days_after, days_before):
    inputs_list = []
    for site in site_list:
        site_poly,site_date = get_site_coordinates(sitetable, site)
        date_range = set_date_range(site_date, days_before, days_after) 
        # it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
        polygon = SDS_tools.smallest_rectangle(site_poly)
        # date range
        dates = date_range
        # satellite missions ['L5','L7','L8','L9','S2']
        sat_list = ['S2']
        # choose Landsat collection 'C01' or 'C02'
        collection = 'C02'
        # name of the site
        sitename = site
        # directory where the data will be stored
        filepath = os.path.join(os.getcwd(), 'data')
        # put all the inputs into a dictionnary
        inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath,
         'landsat_collection': collection}

        inputs['include_T2'] = False# inputs['include_T2'] = True
        metadata = SDS_download.check_images_available(inputs);
        if(len(metadata[0]['S2']) > 0):
            metadata = SDS_download.retrieve_images(inputs)
        inputs_list.append(inputs)  
    return inputs_list

def batch_check(sitetable, site_list, days_after, days_before):
    inputs_list = []
    for site in site_list:
        site_poly,site_date = get_site_coordinates(sitetable, site)
        date_range = set_date_range(site_date, days_before, days_after) 
        # it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
        polygon = SDS_tools.smallest_rectangle(site_poly)
        # date range
        dates = date_range
        # satellite missions ['L5','L7','L8','L9','S2']
        sat_list = ['S2']
        # choose Landsat collection 'C01' or 'C02'
        collection = 'C02'
        # name of the site
        sitename = site
        # directory where the data will be stored
        filepath = os.path.join(os.getcwd(), 'data')
        # put all the inputs into a dictionnary
        inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath,
         'landsat_collection': collection}

        inputs['include_T2'] = False# inputs['include_T2'] = True
        metadata = SDS_download.check_images_available(inputs)
        inputs_list.append(inputs)  
    return inputs_list

In [5]:
site_names = sitetable['Island Name'].values
site_names

array(['Dhihdhoo', 'Kulhudhuffushi', 'Funadhoo', 'Komandoo ', 'Hinnavaru',
       'Naifaru', 'Eydhafushi', 'Thulhaadhoo', 'Thulusdhoo', 'Huraa',
       'Hinmafushi', 'Hulhumaale', 'Maale', "Vilin'gili", 'Gulhifalhu',
       'Thilafushi', 'Maafushi', 'Nilandhoo', 'Meedhoo', 'Maduvvari',
       'Muli', 'Vilufushi', 'Madifushi', 'Guraidhoo', "Vilin'gili",
       'Dhaandhoo', 'Thinadhoo', 'Gahdhoo', 'Hithadhoo', 'Feydhoo'],
      dtype=object)

In [6]:
site_names = ['Hinnavaru','Eydhafushi', 'Thulhaadhoo', 'Thulusdhoo', 'Huraa',
       'Hinmafushi', 'Hulhumaale', 'Maale', "Vilin'gili", 'Gulhifalhu','Thilafushi', 'Maafushi', 'Nilandhoo', 'Meedhoo', 'Maduvvari',
       'Muli', 'Vilufushi', 'Madifushi', 'Guraidhoo', "Vilin'gili", 'Dhaandhoo', 'Thinadhoo', 'Gahdhoo', 'Hithadhoo', 'Feydhoo']

In [7]:
site_names = site_names[0:1]

In [13]:
inputs_list = batch_check(sitetable, site_names, 365*3, 0)

Number of images available between 2014-06-01 and 2017-05-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 82 images
  Total to download: 82 images


In [ ]:
#inputs_list = batch_download(sitetable, site_names, 365*3, 0)

In [19]:
def remove_duplicates(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if "dup" in file:
                os.remove(os.path.join(root, file))
                print(f"removed {file}")

In [ ]:
remove_duplicates('data/'+site_names[0])

In [22]:
# settings for the sand contour mapping
settings = { 
    # general parameters:
    'cloud_thresh': 0.2,        # threshold on maximum cloud cover
    'dist_clouds': 30,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold        
    'check_detection_sand_poly': False, # if True, uses sand polygon for detection and shows user for validation 
    'save_figure': False,               # if True, saves a figure showing the mapped shoreline for each image
    # add the inputs defined previously
    'inputs': inputs_list[0],
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 3,        # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 25,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 50,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': True,  # switch this parameter to True if sand pixels are masked (in black) on many images
    'sand_color': 'bright',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    's2cloudless_prob': 60,     # probability threshold to identify cloudy pixels in the s2cloudless mask
}
metadata = SDS_download.check_images_available(inputs_list[0])
metadata
#SDS_preprocess.save_jpg(metadata, settings)

Number of images available between 2014-06-01 and 2017-05-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 82 images
  Total to download: 82 images


({'S2': [{'type': 'Image',
    'bands': [{'id': 'B1',
      'data_type': {'type': 'PixelType',
       'precision': 'int',
       'min': 0,
       'max': 65535},
      'dimensions': [1830, 1830],
      'crs': 'EPSG:32643',
      'crs_transform': [60, 0, 300000, 0, -60, 700020]},
     {'id': 'B2',
      'data_type': {'type': 'PixelType',
       'precision': 'int',
       'min': 0,
       'max': 65535},
      'dimensions': [10980, 10980],
      'crs': 'EPSG:32643',
      'crs_transform': [10, 0, 300000, 0, -10, 700020]},
     {'id': 'B3',
      'data_type': {'type': 'PixelType',
       'precision': 'int',
       'min': 0,
       'max': 65535},
      'dimensions': [10980, 10980],
      'crs': 'EPSG:32643',
      'crs_transform': [10, 0, 300000, 0, -10, 700020]},
     {'id': 'B4',
      'data_type': {'type': 'PixelType',
       'precision': 'int',
       'min': 0,
       'max': 65535},
      'dimensions': [10980, 10980],
      'crs': 'EPSG:32643',
      'crs_transform': [10, 0, 300000, 0, -